In [41]:
import os
import re
import time
import pickle
import pandas as pd
import getpass
from tqdm import tqdm
from collections import defaultdict
from IPython.display import clear_output

from langchain.docstore.document import Document
from langchain.chains import RetrievalQA, LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.embeddings import ClovaXEmbeddings
from langchain_community.chat_models import ChatClovaX
from pymilvus import connections, utility
from langchain_community.vectorstores.milvus import Milvus

In [20]:
##############################################
# Milvus 연결 및 환경 설정
##############################################

connections.connect(alias="default", host="localhost", port="19530")
print(utility.get_server_version())
print(connections.has_connection(alias="default"))

os.environ["NCP_CLOVASTUDIO_API_KEY"] = getpass.getpass("Test Key 입력: ")
os.environ["NCP_CLOVASTUDIO_API_URL"] = "https://clovastudio.stream.ntruss.com/"

v2.4.0
True


### 임베딩 pkl 합치기

In [ ]:
# # 임베딩 파일 경로
# folder_path = r"C:\Kill_the_RAG\Project\Aiffel_final_project\Code\Data\example"

# # merge 파일 리스트
# file_names = [f"embedding_category_{i}.pkl" for i in range(1, 5)]

# merged_embeddings = []
# merged_metadata = []

# # 각 파일을 순회하면서 데이터를 합침
# for file_name in file_names:
#     file_path = os.path.join(folder_path, file_name)
#     print(f"파일 로드 중: {file_path}")
#     with open(file_path, "rb") as f:
#         data = pickle.load(f)
#     # "embeddings"와 "metadata" 키가 존재한다면 리스트에 추가
#     embeddings = data.get("embeddings", [])
#     metadata = data.get("metadata", [])
#     merged_embeddings.extend(embeddings)
#     merged_metadata.extend(metadata)

# # 합쳐진 데이터를 딕셔너리로 구성
# merged_data = {"embeddings": merged_embeddings, "metadata": merged_metadata}

# # 저장할 파일 경로
# output_file = os.path.join(folder_path, "embedding_Category.pkl")
# with open(output_file, "wb") as f:
#     pickle.dump(merged_data, f)

# print(f"{output_file}에 merge 완료")

파일 로드 중: C:\Kill_the_RAG\Project\Aiffel_final_project\Code\Data\example\embedding_category_1.pkl
파일 로드 중: C:\Kill_the_RAG\Project\Aiffel_final_project\Code\Data\example\embedding_category_2.pkl
파일 로드 중: C:\Kill_the_RAG\Project\Aiffel_final_project\Code\Data\example\embedding_category_3.pkl
파일 로드 중: C:\Kill_the_RAG\Project\Aiffel_final_project\Code\Data\example\embedding_category_4.pkl
C:\Kill_the_RAG\Project\Aiffel_final_project\Code\Data\example\embedding_Category.pkl에 merge 완료


In [21]:
# 데이터 로드 및 전처리
df = pd.read_csv("C:\\Kill_the_RAG\\Project\\Aiffel_final_project\\Code\\Data\\sampled_data.csv")
df = df.drop(columns=['Unnamed: 0'])
df = df.sample(n=2000, random_state=2025)

metadata_mapping = {
    'ISBN': 'ISBN',
    '페이지': 'page',
    '가격': 'price',
    '제목': 'title',
    '저자': 'author'
}

vector_doc_mapping = {
    '제목': 'title',
    '분류': 'category',
    '저자': 'author',
    '저자소개': 'author_intro',
    '책소개': 'book_intro',
    '목차': 'table_of_contents',
    '출판사리뷰': 'publisher_review',
    '추천사': 'recommendation'
}

metadata_columns = ['ISBN', '페이지', '가격', '제목', '저자']
vector_doc_columns = ['제목', '분류', '저자', '저자소개', '책소개', '목차', '출판사리뷰', '추천사']

In [22]:
def split_text(text, chunk_size=1000, overlap=100):
    if text is None or pd.isnull(text):
        return [""]
    chunks = []
    for i in range(0, len(text), chunk_size - overlap):
        chunks.append(text[i:i + chunk_size])
    return chunks

In [23]:
# RAG DB 생성
RAG_DB = []
for index, row in df.iterrows():
    doc_text = ""
    for col in vector_doc_columns:
        value = row.get(col, "")
        if pd.isnull(value):
            value = ""
        eng_col = vector_doc_mapping.get(col, col)
        doc_text += f"{eng_col} : {value}\n"
    chunks = split_text(doc_text)
    metadata = {}
    for col in metadata_columns:
        value = row.get(col, None)
        eng_col = metadata_mapping.get(col, col)
        metadata[eng_col] = value
    for chunk in chunks:
        RAG_DB.append({
            'text': chunk,
            'metadata': metadata
        })

documents = [Document(page_content=entry['text'], metadata=entry['metadata']) for entry in RAG_DB]

In [9]:
# 임베딩 모델 및 LLM 초기화
ncp_embeddings = ClovaXEmbeddings(model="bge-m3")
llm_clova = ChatClovaX(model="HCX-003",max_tokens=2048)

In [24]:
embedding_file = "C:\\Kill_the_RAG\\Project\\Aiffel_final_project\\Code\\Data\\ncp_bge_m3_embeddings.pkl"
if os.path.exists(embedding_file):
    with open(embedding_file, "rb") as f:
        saved_data = pickle.load(f)
    all_text_embedding_pairs = saved_data["embeddings"]
    all_metadata_list = saved_data["metadata"]
    print("임베딩 데이터 불러오기")
else:
    all_text_embedding_pairs = []
    all_metadata_list = []
    batch_size = 2000
    for i, doc in enumerate(tqdm(documents, desc="NCP 임베딩(bge-m3) 처리 중", unit="document")):
        text_chunk = doc.page_content
        embedding = None
        while embedding is None:
            try:
                embedding = ncp_embeddings.embed_query(text_chunk)
            except Exception as e:
                print(f"[에러] 문서 {i} 임베딩 실패: {e}")
                if "429" in str(e):
                    print("[경고] 요청 제한 초과. 5초 대기 후 재시도")
                    time.sleep(5)
                    continue
                embedding = None
        all_text_embedding_pairs.append((text_chunk, embedding))
        all_metadata_list.append(doc.metadata)
        time.sleep(0.8)
        if (i + 1) % batch_size == 0:
            print("배치 처리 2000건 완료, 5초 대기 중...")
            time.sleep(5)
    with open(embedding_file, "wb") as f:
        pickle.dump({"embeddings": all_text_embedding_pairs, "metadata": all_metadata_list}, f)
    print("임베딩 저장 완료")

임베딩 데이터 불러오기


In [25]:
# 메타데이터 매핑 재설정
all_metadata_list_mapped = []
for meta in all_metadata_list:
    mapped_meta = {metadata_mapping.get(key, key): value for key, value in meta.items()}
    all_metadata_list_mapped.append(mapped_meta)

# Milvus 벡터 DB 구축
collection_name = "book_rag_db"
vectorstore = Milvus(
    embedding_function=ncp_embeddings,
    collection_name=collection_name,
    connection_args={"host": "localhost", "port": "19530"},
    auto_id=True
)

texts = [pair[0] for pair in all_text_embedding_pairs]
embeds = [pair[1] for pair in all_text_embedding_pairs]

def precomputed_embed_documents(cls, input_texts):
    if input_texts != texts:
        raise ValueError("ERROR : 입력 텍스트 순서가 사전 계산된 임베딩과 일치하지 않음")
    return embeds
ClovaXEmbeddings.embed_documents = classmethod(precomputed_embed_documents)

vectorstore.add_texts(
    texts=texts,
    metadatas=all_metadata_list_mapped,
    embeddings=embeds
)

[456745779904395490,
 456745779904395491,
 456745779904395492,
 456745779904395493,
 456745779904395494,
 456745779904395495,
 456745779904395496,
 456745779904395497,
 456745779904395498,
 456745779904395499,
 456745779904395500,
 456745779904395501,
 456745779904395502,
 456745779904395503,
 456745779904395504,
 456745779904395505,
 456745779904395506,
 456745779904395507,
 456745779904395508,
 456745779904395509,
 456745779904395510,
 456745779904395511,
 456745779904395512,
 456745779904395513,
 456745779904395514,
 456745779904395515,
 456745779904395516,
 456745779904395517,
 456745779904395518,
 456745779904395519,
 456745779904395520,
 456745779904395521,
 456745779904395522,
 456745779904395523,
 456745779904395524,
 456745779904395525,
 456745779904395526,
 456745779904395527,
 456745779904395528,
 456745779904395529,
 456745779904395530,
 456745779904395531,
 456745779904395532,
 456745779904395533,
 456745779904395534,
 456745779904395535,
 456745779904395536,
 456745779904

In [26]:
# 기본 Dense Retriever 및 RetrievalQA 체인 (모든 페르소나 기본으로 사용; Science의 경우 추후 ensemble 추가 가능)
dense_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
dpr_qa_chain = RetrievalQA.from_chain_type(
    llm=llm_clova,
    retriever=dense_retriever,
    return_source_documents=True
)

In [27]:
def extract_field(text, field_name):
    pattern = rf"{re.escape(field_name)}\s*:\s*(.*)"
    match = re.search(pattern, text)
    return match.group(1).strip() if match else ""

MIN_INFO_LENGTH = 10

### 공용 시스템 프롬프트

In [42]:
##############################################
# 공통 프롬프트 템플릿 정의 (역할만 치환)
##############################################
# ※ 주의: 아래에서 {history}, {query}, {fallback}, {score}는 LLMChain에서 나중에 치환될 변수이므로
#     Python의 format() 시에는 이중 중괄호로 작성해야 합니다.
#     또한 출력 형식에 포함된 리터럴 중괄호들은 quadruple braces를 사용합니다.

common_multi_turn_prompt = """
[대화 맥락]
사용자 대화 내역:
{{history}}
사용자의 최신 질문: "{{query}}"

[역할 및 목표]
{role_instructions}

[MultiChat 출력 형식 - 반드시 아래 내용만 출력]
검색 확률: {{{{숫자, 0~1 사이 (예: 0.75)}}}}
기본 검색 쿼리: "{{{{검색 쿼리, 예: 흥미로운 조선시대 역사적 사건을 다룬 한국사 서적}}}}"
추가 질문: "{{{{추가 질문, 정보 보완 필요 시 구체적으로; 충분하면 빈 문자열}}}}"

[출력 예시]
검색 확률: 0.75
기본 검색 쿼리: "이해하기 쉬운 쉬운 역사책"
추가 질문: "혹시 어느 시대 역사책을 찾고 있나요?"
"""

common_final_query_prompt = """
[전체 대화 요약]
{{history}}

[이전 기본 검색 쿼리]
{{fallback}}

[이전 검색 확률]
{{score}}

[역할 및 목표]
{role_instructions}

[최종 출력 형식 - 반드시 아래 내용만 출력]
검색 확률: {{score}}
검색 쿼리: "{{{{최종 검색 쿼리}}}}"
책 제목: "{{{{책 제목}}}}"
저자: "{{{{저자}}}}"
추천 이유: "{{{{추천 이유, 정보 부족 시 '추천 이유 정보 없음'}}}}"

[출력 예시]
검색 확률: "0.75"
검색 쿼리: "흥미로운 조선시대 역사적 사건과 인물을 다룬 한국사 서적"
책 제목: "조선 왕조 실록"
저자: "홍길동"
추천 이유: "조선시대의 정치, 문화, 경제적 요소를 균형 있고 쉽게 반영하여 심층적 역사 이해에 도움을 줄 수 있음"
"""

# 각 페르소나별 역할 문구
literature_role = (
    "너는 감성적이고 문학적인 도서 추천 챗봇이다. 대화를 통해 사용자의 선호하는 장르, 작가, 시대를 파악하고, "
    "감성적 표현을 활용하여 책을 추천해라."
)

science_role = (
    "너는 정확하고 논리적인 과학/기술 도서 추천 챗봇이다. 대화를 통해 사용자의 관심 분야(컴퓨터, 생물, 물리 등)와 "
    "난이도(입문, 심화)를 파악하여, 명확하고 구체적인 정보를 바탕으로 책을 추천해라."
)

### Modular RAG 파이프라인

In [43]:
##############################################
# 모듈화된 RAG 파이프라인 클래스 설계
##############################################

class BaseRAGPipeline:
    def __init__(self, config, llm, retriever, qa_chain, documents):
        self.config = config
        self.llm = llm
        self.retriever = retriever
        self.qa_chain = qa_chain
        self.documents = documents
        self.query_history = []
        self.user_preferences = defaultdict(list)
        
        # 공통 프롬프트에서 role_instructions만 치환 (나머지 {{history}}, {{query}} 등은 그대로 남음)
        self.multi_turn_chain = LLMChain(
            llm=self.llm,
            prompt=PromptTemplate(
                input_variables=["history", "query"],
                template=common_multi_turn_prompt.format(role_instructions=config["role_instructions"])
            )
        )
        self.final_query_chain = LLMChain(
            llm=self.llm,
            prompt=PromptTemplate(
                input_variables=["history", "fallback", "score"],
                template=common_final_query_prompt.format(role_instructions=config["role_instructions"])
            )
        )
    
    def robust_parse_llm_response(self, response_text):
        score_match = re.search(r"검색\s*확률\s*[:：]\s*([\d\.]+)", response_text)
        search_score = float(score_match.group(1)) if score_match else None

        query_match = re.search(r"기본\s*검색\s*쿼리\s*[:：]\s*\"([^\"]+)\"", response_text)
        search_query = query_match.group(1).strip() if query_match else None

        follow_match = re.search(r"추가\s*질문\s*[:：]\s*\"([^\"]*)\"", response_text)
        follow_up_question = follow_match.group(1).strip() if follow_match else ""

        return search_score, search_query, follow_up_question

    def generate_answer(self, query):
        result = self.qa_chain.invoke(query)
        source_docs = result['source_documents']

        retrieved_isbns = set()
        for doc in source_docs:
            isbn = doc.metadata.get("ISBN")
            if isbn:
                retrieved_isbns.add(isbn)

        aggregated_docs = []
        for isbn in retrieved_isbns:
            book_docs = [doc for doc in self.documents if doc.metadata.get("ISBN") == isbn]
            if not book_docs:
                continue
            aggregated_text = "\n".join([doc.page_content for doc in book_docs])
            aggregated_docs.append(Document(page_content=aggregated_text, metadata=book_docs[0].metadata))

        formatted_answers = []
        for doc in aggregated_docs:
            metadata = doc.metadata
            title = metadata.get("title") or extract_field(doc.page_content, "title")
            author = metadata.get("author") or extract_field(doc.page_content, "author")
            extra_field = ""
            if self.config.get("persona") == "Science":
                extra_field = metadata.get("year") or extract_field(doc.page_content, "year")
            publisher_review = extract_field(doc.page_content, "publisher_review")
            book_intro = extract_field(doc.page_content, "book_intro")

            if publisher_review and book_intro:
                combined_info = publisher_review + "\n" + book_intro
            elif publisher_review:
                combined_info = publisher_review
            elif book_intro:
                combined_info = book_intro
            else:
                combined_info = ""

            if not combined_info or len(combined_info.strip()) < MIN_INFO_LENGTH:
                reason = "추천 이유 정보 없음"
            else:
                reason_prompt = (
                    f"다음 정보를 참고하여, 이 책이 추천되는 이유를 간결하고 명확하게 요약해라. "
                    f"사용자 선호도에 기반하여 책의 특징이나 강점을 중심으로 설명해라. 만약 선호도를 모르겠다면, '추천 이유 정보 없음'을 출력해라.\n\n정보:\n{combined_info}"
                )
                reason_response = self.llm.invoke(reason_prompt)
                generated_reason = reason_response.text().strip()
                if not generated_reason or len(generated_reason) < 10 or "추천 이유 정보 없음" in generated_reason:
                    reason = "추천 이유 정보 없음"
                else:
                    reason = generated_reason

            if self.config.get("persona") == "Science":
                formatted = f"{title}\n{author}\n출판년도: {extra_field if extra_field else '정보 없음'}\n추천 이유: {reason}"
            else:
                formatted = f"{title}\n{author}\n추천 이유: {reason}"
            formatted_answers.append(formatted)

        answer = "\n\n".join(formatted_answers)
        return answer, None

    def search_and_generate_answer(self, query):
        while True:
            query_summary = "\n".join(self.query_history[-5:])
            prompt_vars = {"history": query_summary, "query": query}
            search_decision_dict = self.multi_turn_chain.invoke(prompt_vars)
            response_text = search_decision_dict["text"].strip()
            print("\n[🔍 LLM 응답 확인]\n", response_text)

            search_score, base_search_query, follow_up_question = self.robust_parse_llm_response(response_text)
            print(f"\n[디버그] 파싱 결과: 검색 확률={search_score}, 기본 검색 쿼리='{base_search_query}', 추가 질문='{follow_up_question}'")

            if search_score is None:
                print("\n[LLM 응답 파싱 실패: 추가 정보 필요]")
                extra_info = input("추가 정보를 입력해주세요: ")
                self.query_history.append(f"사용자(추가): {extra_info}")
                query = f"{query} {extra_info}"
                continue

            if search_score >= 0.8 and base_search_query:
                final_search_query = self.final_query_chain.invoke({
                    "history": "\n".join(self.query_history),
                    "fallback": base_search_query,
                    "score": search_score
                })["text"].strip()
                print(f"\n[최종 검색 쿼리 생성]: {final_search_query}")

                answer, sources = self.generate_answer(final_search_query)
                if sources:
                    book_info = "\n".join([f"- {title}" for title in sources])
                    answer_with_info = f"{answer}\n\n[책 정보]\n{book_info}"
                    print("\n[책 정보]\n", book_info)
                else:
                    answer_with_info = answer
                return answer_with_info

            if follow_up_question:
                print(f"\n[보충 질문: {follow_up_question}]")
                self.query_history.append(f"AI: {follow_up_question}")
                user_response = input("\n사용자 응답: ")
                self.query_history.append(f"사용자: {user_response}")
                if "장르" in follow_up_question or "작가" in follow_up_question or "시대" in follow_up_question:
                    self.user_preferences["literature"].append(user_response)
                else:
                    self.user_preferences["misc"].append(user_response)
                print("\n[사용자 선호도 업데이트 완료!]")
                query = f"{query} {follow_up_question} {user_response}"
                continue

            if search_score < 0.8 or not base_search_query:
                print("\n[검색 확률 낮거나 검색 쿼리 없음: 추가 정보 필요]")
                extra_info = input("추가 정보를 입력해주세요: ")
                self.query_history.append(f"사용자(추가): {extra_info}")
                query = f"{query} {extra_info}"
                continue

    def interactive_multi_turn_qa(self):
        while True:
            clear_output(wait=True)
            print(f"멀티턴 AI 기반 책 추천 시스템 - {self.config.get('persona', 'Default')} 페르소나 (종료하려면 'quit' 입력)")
            print("-" * 50)
            query = input("질문을 입력하세요: ")
            if query.lower() == 'quit':
                print("\n[대화 저장 중...]")
                print("대화 저장 완료")
                break
            self.query_history.append(f"사용자: {query}")
            answer = self.search_and_generate_answer(query)
            print("\n[AI의 답변]")
            print(answer)
            self.query_history.append(f"AI: {answer}")
            input("\n-> 계속하려면 Enter를 누르세요...")

In [44]:
##############################################
# 예술/문학 페르소나 전용 파이프라인 클래스
##############################################

class LiteratureRAGPipeline(BaseRAGPipeline):
    def __init__(self, llm, retriever, qa_chain, documents):
        config = {
            "persona": "Literature",
            "role_instructions": literature_role
        }
        super().__init__(config, llm, retriever, qa_chain, documents)

##############################################
# 과학/기술 페르소나 전용 파이프라인 클래스
##############################################

class ScienceRAGPipeline(BaseRAGPipeline):
    def __init__(self, llm, retriever, qa_chain, documents):
        config = {
            "persona": "Science",
            "role_instructions": science_role
        }
        super().__init__(config, llm, retriever, qa_chain, documents)

In [45]:
##############################################
# 메인 실행 함수: 페르소나 선택 후 대화 실행
##############################################

def main():
    print("페르소나 선택:")
    print("1. 예술/문학")
    print("2. 과학/기술")
    choice = input("원하는 페르소나 번호를 입력하세요 (1 또는 2): ").strip()
    
    if choice == "1":
        pipeline = LiteratureRAGPipeline(llm_clova, dense_retriever, dpr_qa_chain, documents)
    elif choice == "2":
        pipeline = ScienceRAGPipeline(llm_clova, dense_retriever, dpr_qa_chain, documents)
    else:
        print("잘못된 선택입니다. 기본 예술/문학 페르소나로 실행합니다.")
        pipeline = LiteratureRAGPipeline(llm_clova, dense_retriever, dpr_qa_chain, documents)
    
    pipeline.interactive_multi_turn_qa()

if __name__ == "__main__":
    main()

멀티턴 AI 기반 책 추천 시스템 - Science 페르소나 (종료하려면 'quit' 입력)
--------------------------------------------------

[🔍 LLM 응답 확인]
 검색 확률 : 0.8 
기본 검색 쿼리 : "AI 에이전트 관련 기술서적"
추가 질문 : "어떤 분야의 AI 에이전트에 대해 알고 싶으신가요?(예: 자연어 처리, 강화학습 등)"

[디버그] 파싱 결과: 검색 확률=0.8, 기본 검색 쿼리='AI 에이전트 관련 기술서적', 추가 질문='어떤 분야의 AI 에이전트에 대해 알고 싶으신가요?(예: 자연어 처리, 강화학습 등)'

[최종 검색 쿼리 생성]: 검색 확률: 0.8
검색 쿼리: "AI 에이전트 관련 기술서적"
책 제목: "인공지능 시대의 비즈니스 전략"
저자: 정도희 
출판사 : 더퀘스트
추천 이유 : 인공지능이 어떻게 기업의 생산성 향상 , 새로운 비즈니스 모델 창출에 기여할 수 있는지 다양한 사례를 통해 설명하며, 특히 AI 에이전트가 어떤 역할을 할 수 있는지 자세히 다루고 있습니다. 

책 제목 : "인간과 협업하는 로봇과 에이전트"
저자 : 강현철 외 2명
출판사 : 홍릉과학출판사
추천 이유 : 이 책은 인간과 협업하는 로봇과 에이전트들의 설계 방법과 구현 방안들을 제시함으로써 지능로봇 시스템 개발에 필요한 기초 지식을 습득할 수 있도록 안내한다. 또한, 각 장에서 소개되는 예제 코드들은 독자들이 직접 실행해 볼 수 있도록 제공되어 있어 학습 효과를 높일 수 있다.

[AI의 답변]
AI시대 사람의 조건 휴탈리티
박정열 지음
출판년도: 정보 없음
추천 이유: 이 책은 "사람은 많은데 인재는 없다?"라는 질문에 대한 답을 제시해주는 책으로, 인재란 누구이며 갖추어야 할 본질적인 역량은 무엇인지 알려줍니다. 또한, 인공지능 시대에 인간의 생존력은 어디에서 찾아야 하는지에 대한 고민을 담고 있습니다.

인공지능이라는 슈퍼 기계가 등장하면서 인간의 가치가 저평가되고 있는 상황에서, 이 책은 인간의

KeyboardInterrupt: Interrupted by user